# Title
Intro

In [ ]:
# Import modules
%matplotlib inline


import os
import sys
import matplotlib

import matplotlib.pyplot as plt

from osgeo import gdal
from functools import partial
from pathlib import Path

gdal.UseExceptions()

In [ ]:
src = Path(__file__).parent.joinpath("src").as_posix()

sys.path.insert(0, src)

from parallelprocessor import ParallelProcessor

Set up paths

In [ ]:
examples = Path(__file__)

data = examples.parent.joinpath("data")

tiles = data.joinpath("tiles")

if not tiles.exists():
    tiles.mkdir()

geotiff = data.joinpath("geotiffs", "wellington-03m-rural-aerial-photos-2021.tif")

geotiff_path = geotiff.as_posix()

tile_base = tiles.joinpath(geotiff.stem + "_{}_{}.ecw").as_posix()

Create pixel offset windows

In [ ]:

src = gdal.Open(fn, 0)

width = src.RasterXSize
height = src.RasterYSize

x_0 = 0
y_0 = 0
x_step = 256
y_step = 256

f_x_step = lambda i, width, x_step: x_step if i + x_step <= width else width - i
f_y_step = lambda j, height, y_step: y_step if j + y_step <= height else height - j

f = partial(f_x_step, width=width, x_step=x_step)
g = partial(f_y_step, height=height, y_step=y_step)

windows = [
    (
        tile_base.format(i, j), 
        [i, j, f(i), g(j)],
    )
    for i in range(x_0, width, x_step)
    for j in range(y_0, height, y_step)
]

src = None

Create gdal_translate commands

In [ ]:
func = gdal.Translate

argument_list = [
    {
        "process_id": tile_path,
        "func_args": (tile_path, geotiff_path),
        "func_kwargs" = {
            "options": gdal.TranslateOptions(
                format="ECW",
                srcWin=window,
                creationOptions=["TARGET=100"]
            )
        }
    }
]

In [ ]:
parallel_processor = ParallelProcessor(func)

for argument_dict in argument_list:
    parallel_processor.add_argument(*argument_dict)

In [ ]:
parallel_processor.run(progressbar=True, timeout=60*10)

In [ ]:
results = parallel_processor.results

Display results

In [ ]:
def dispay_raster(raster):

    src = gdal.Open(raster, 0)

    bands = (
        src.GetRasterBand(i + 1).ReadAsArray()
        for i in range(src.RasterCount)
    )
    
    img = np.dstack(bands)

    f = plt.figure()

    plt.imshow(img)

    plt.show()

    return f

In [ ]:
for ecw, status in results.items():
    if status:
        _ = dispay_raster(ecw)